In [1]:
pip install scgraph>=0.3.0

In [5]:
from scgraph import Graph
from scgraph.data import marnet_data

my_graph = Graph(data=marnet_data)
# Get the shortest path between
output = my_graph.get_shortest_path(
    origin={"latitude": 31.23,"longitude": 121.47},
    destination={"latitude": 32.08,"longitude": -81.09}
)
print(output['length'])
print(str([[i['latitude'],i['longitude']] for i in output['path']]))

19596.73547212812
[[31.23, 121.47], [30.732, 121.827], [30.835, 122.678], [31.3, 122.9], [33.546, 126.54], [34.2, 127.6], [35.0, 129.2], [38.211, 134.084], [40.0, 137.266], [41.2, 139.4], [41.9, 141.25], [41.9, 143.3], [43.2, 146.0], [46.098, 153.352], [46.196, 153.688], [47.445, 157.933], [48.11, 160.194], [48.325, 160.925], [48.476, 161.437], [50.0, 169.019], [50.154, 169.786], [50.238, 170.204], [50.484, 172.355], [50.688, 174.14], [51.3, 179.5], [51.6, -176.7], [51.295, -171.555], [51.097, -168.206], [51.08, -168.081], [50.751, -165.623], [50.373, -162.794], [50.0, -160.0], [49.171, -154.535], [48.951, -153.456], [48.57, -151.59], [48.245, -150.001], [48.097, -149.277], [47.633, -147.484], [46.797, -144.254], [46.635, -143.741], [45.457, -140.003], [45.294, -139.486], [44.112, -136.329], [43.606, -134.977], [43.486, -134.7], [42.494, -132.411], [42.442, -132.291], [40.889, -130.834], [40.0, -130.0], [39.492, -129.45], [36.659, -126.379], [36.481, -126.187], [36.12, -125.796], [35.4